In [107]:
# Importing necessary libraries
import pandas as pd

## Data Loading
The following datasets are loaded into the notebook:
1. **Product Catalog**: Contains detailed information about each product.
2. **Transactions**: Includes all purchase transactions with customer and product information.
3. **Product Category Map**: Maps products to their respective categories.
4. **Test Data**: Dataset for final predictions.

The initial few rows of each dataset are displayed to confirm successful loading and understand the structure of the data.


In [108]:
# Loading datasets
product_catalog = pd.read_csv('../data/product_catalog.csv')
transactions = pd.read_csv('../data//transactions.csv')
product_category_map = pd.read_csv('../data//product_category_map.csv')
test = pd.read_csv('../data//test.csv')

## Data Preprocessing

In [109]:
# 1. **Convert `purchase_date` to datetime format**
# Ensures date-related calculations can be performed.
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])

# 2. **Group by `customer_id` and `product_id`**
# Aggregates key metrics such as first purchase, last purchase, and total purchase count.
grouped = transactions.groupby(['customer_id', 'product_id']).agg(
    first_purchase=('purchase_date', 'min'),
    last_purchase=('purchase_date', 'max'),
    total_purchases=('purchase_date', 'count')
).reset_index()

# 3. **Calculate second last purchase date**
# Helper function to find the second last purchase date for each customer-product pair.
def get_second_last_purchase(dates):
    if len(dates) > 1:
        return sorted(dates)[-2]  # Return the second last date if more than one purchase exists.
    return pd.NaT  # Return NaT (Not a Time) if there is only one purchase.

second_last_purchase = (
    transactions.groupby(['customer_id', 'product_id'])['purchase_date']
    .apply(get_second_last_purchase)
    .reset_index(name='second_last_purchase')
)

# Merge the second last purchase date into the grouped DataFrame.
grouped = grouped.merge(second_last_purchase, on=['customer_id', 'product_id'], how='left')

# 4. **Calculate average days between purchases**
# Computes the average interval between purchases for each customer-product pair.
grouped['average_days_between_purchases'] = (
    (grouped['last_purchase'] - grouped['first_purchase']).dt.days /
    (grouped['total_purchases'] - 1)
).fillna(0)  # Fill NaN with 0 for cases with only one purchase.

# 5. **Filter pairs with at least two purchases**
# Retains only customer-product pairs with 2 or more total purchases.
grouped = grouped[grouped['total_purchases'] >= 2]

# 6. **Calculate week codes**
# Calculates the number of weeks since a reference date for the last purchase.
reference_date = pd.Timestamp('2020-12-25')
grouped['week_code'] = ((grouped['last_purchase'] - reference_date).dt.days // 7)
grouped['week_code'] = grouped['week_code'].replace(5, 4)  # Adjust outliers in week codes.

# 7. **Calculate product popularity**
# Maps product popularity based on the total number of purchases.
grouped['product_id_count'] = grouped['product_id'].map(grouped['product_id'].value_counts())
grouped['popularity'] = pd.qcut(
    grouped['product_id_count'],
    q=10,  # Divide into 10 quantile bins
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
).astype(int)

# 8. **Apply additional filters**
# Filters based on logical conditions for purchase count and week codes.
valid_filter = (
    (grouped['total_purchases'] > 2) & (grouped['week_code'] > 0) |
    (grouped['total_purchases'] > 1) & (grouped['week_code'] <= 0)
)
grouped = grouped[valid_filter]

# 9. **Calculate predictions and adjust last purchase dates**
# Predictions based on week codes, and adjusts the last purchase date using the second last purchase if applicable.
grouped['prediction'] = grouped['week_code'].apply(lambda x: x if x > 0 else 0)
grouped['last_last_purchase'] = grouped.apply(
    lambda row: row['second_last_purchase'] if row['prediction'] != 0 else row['last_purchase'], axis=1
)

# 10. **Calculate second week code**
# Computes week code for the second last purchase date.
grouped = grouped[grouped['second_last_purchase'] < '2021-01-01']
grouped['week_code_2'] = ((grouped['second_last_purchase'] - reference_date).dt.days // 7)

# 11. **Final cleanup**
# Drops unnecessary columns and renames 'week_code_2' for clarity.
final_df = grouped.drop(columns=[
    'product_id', 'customer_id', 'first_purchase', 'last_purchase',
    'total_purchases', 'second_last_purchase', 'week_code', 'product_id_count', 'last_last_purchase'
])
final_df['popularity'] = final_df['popularity'].astype(int)
final_df.rename(columns={'week_code_2': 'last_purchase_week_code'}, inplace=True)

# Output the final DataFrame
train_set = final_df

# Save the prepared test set to a CSV file
train_set.to_csv('../processed_data/preprocessed_train_set.csv', index=False)

print("Train set has been saved successfully as 'preprocessed_test_set.csv'")


Train set has been saved successfully as 'preprocessed_test_set.csv'
